In [ ]:
import scvelo as scv


In [ ]:
scv.set_figure_params()

In [ ]:
adata = scv.read("/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_scVelo_annData_object.h5ad", cache=True)

In [ ]:
adata

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)

In [ ]:
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

In [ ]:
scv.tl.recover_dynamics(adata) ## model

In [ ]:
scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap', color="modules", palette = ["gray","blue","brown","pink","yellow"],
                         alpha = 0.4, legend_loc = 'none', figsize = [8,6],       
                         save='/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Modules_Stream_Embedding2.png')

In [ ]:
scv.pl.proportions(adata, groupby="modules", save='/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Modules_Proportions.pdf')

In [ ]:
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, 
               save='/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Modules_latent-time.pdf')

In [ ]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:300]
scv.pl.heatmap(adata, var_names=top_genes, sortby='latent_time', col_color='modules', 
               palette = ["black","blue","brown","pink","yellow"], n_convolve=100, 
               save='/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Modules_latent-time_HeatMap.pdf')

In [ ]:
adata.var

In [ ]:
#Output heatmap table
import numpy as np
import scipy
import pandas as pd
var_names=top_genes
tkey="latent_time"
xkey="Ms"
n_convolve=100
sort=True

time = adata.obs[tkey].values
time = time[np.isfinite(time)]
X = (adata[:, var_names].layers[xkey]
if xkey in adata.layers.keys()
else adata[:, var_names].X
)

if scipy.sparse.issparse(X): X = X.A

df = pd.DataFrame(X[np.argsort(time)], columns=var_names)
if n_convolve is not None:
	weights = np.ones(n_convolve) / n_convolve
	for gene in var_names:
		try:
			df[gene] = np.convolve(df[gene].values, weights, mode="same")
		except:
			pass  # e.g. all-zero counts or nans cannot be convolved
if sort:
	max_sort = np.argsort(np.argmax(df.values, axis=0))
	df = pd.DataFrame(df.values[:, max_sort], columns=df.columns[max_sort])


outFILE="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/df.timeSeries.heatmap.sorted.csv"
df.to_csv(outFILE, header=True,index=False)

In [ ]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index
scv.pl.scatter(adata, basis=top_genes[:15], ncols=5, frameon=False, 
              color="modules", palette = ["gray","blue","brown","pink","yellow"])

In [ ]:
#Cluster-specific top-likelihood genes
scv.tl.rank_dynamical_genes(adata, groupby='modules')
df = scv.get_df(adata, 'rank_dynamical_genes/names')
df.head(10)

In [ ]:
outFILE="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Module-specific_top-likelihood_genes.csv"
df.to_csv(outFILE, header=True,index=False)

In [ ]:
kwargs = dict(frameon=False, size=10, linewidth=1.5,
              add_outline='blue, yellow, brown, pink')
genes_use = ['BICC1','TNIK','DCDC2','ARHGAP29']
outFILE="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Module-specific_top-likelihood_genes_phase-portraits-gene-set1a.pdf"
scv.pl.scatter(adata, basis=genes_use, ncols=4, color="modules", 
               palette = ["gray","blue","brown","pink","yellow"], **kwargs, save = outFILE)
outFILE="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Module-specific_top-likelihood_genes_phase-portraits-gene-set1b.pdf"
scv.pl.scatter(adata, x='latent_time', ncols=4, y=genes_use, color="modules", 
               palette = ["gray","blue","brown","pink","yellow"], **kwargs, save = outFILE)

In [ ]:
kwargs = dict(frameon=False, size=10, linewidth=1.5,
              add_outline='blue, yellow, brown, pink')
genes_use = ['ITGB6','ITGB8','PROM1','HIF1A']
outFILE="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Module-specific_top-likelihood_genes_phase-portraits-gene-set2a.pdf"
scv.pl.scatter(adata, basis=genes_use, ncols=4, color="modules", 
               palette = ["gray","blue","brown","pink","yellow"], **kwargs, save = outFILE)
outFILE="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Module-specific_top-likelihood_genes_phase-portraits-gene-set2b.pdf"
scv.pl.scatter(adata, x='latent_time', ncols=4, y=genes_use, color="modules", 
               palette = ["gray","blue","brown","pink","yellow"], **kwargs, save = outFILE)

In [ ]:
kwargs = dict(frameon=False, size=20, linewidth=1.5)
genes_use = ['TNIK','ARHGAP29','ITGB6','TSPAN1']
outFILE="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Module-specific_top-likelihood_genes_phase-portraits-gene-set5a.pdf"
scv.pl.scatter(adata, basis=genes_use, ncols=4, color="modules", 
               palette = ["gray","blue","brown","pink","yellow"], **kwargs, save = outFILE)
outFILE="/media/Scratch_SSD_Voyager/Blue/KPMP_10X/KPMP_Ref_analysis_12-2020/scVelo/aTAL_Module-specific_top-likelihood_genes_phase-portraits-gene-set5b.pdf"
scv.pl.scatter(adata, x='latent_time', ncols=4, y=genes_use, color="modules", 
               palette = ["gray","blue","brown","pink","yellow"], **kwargs, save = outFILE)